In [ ]:
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

# 生成数据集

In [ ]:
num_inputs = 2  # 是输入特征的数量，这里是 2。
num_examples = 1000  # 是样本的数量，这里是 1000。
true_w = [2, -3.4]
true_b = 4.2
features = torch.randn(num_examples, num_inputs, dtype=torch.float32)  # 生成一个 num_examples 行 num_inputs 列的随机张量
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b  # 是模型的预测输出，等于 true_w 和 features 的加权和加上 true_b。
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float32) # 用来生成一个与 labels 尺寸相同的正态分布噪声，均值为 0，标准差为 0.01。

In [ ]:
print(features[0], labels[0])
# features[0] 是第一个样本的特征，形状为 (2,) 的张量。
# labels[0] 是第一个样本的标签，标量值。

# features的每一行是一个长度为2的向量，而labels的每一行是一个长度为1的向量（标量）。

In [ ]:
from matplotlib_inline.backend_inline import set_matplotlib_formats

def use_svg_display():
    # 用矢量图显示
    set_matplotlib_formats('svg')

def set_figsize(figsize=(3.5, 2.5)):
    use_svg_display()
    # 设置图的尺寸
    plt.rcParams['figure.figsize'] = figsize

set_figsize()
plt.scatter(features[:, 1].numpy(), labels.numpy(), 1);  # plt.scatter 函数用来绘制散点图，其中 features[:, 1] 是 x 轴的数据，labels 是 y 轴的数据。
# 绘制一个散点图，显示特征的第二列与标签之间的关系
# 通过生成第二个特征features[:, 1]和标签 labels 的散点图，可以更直观地观察两者间的线性关系。

# 读取数据

In [ ]:
# 它每次返回batch_size（批量大小）个随机样本的特征和标签。
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)  # 样本的读取顺序是随机的
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)]) # 最后一次可能不足一个batch
        yield  features.index_select(0, j), labels.index_select(0, j)

In [ ]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, y, sep="\n")
    # break

# 初始化模型参数

In [ ]:
# 将权重初始化成均值为0、标准差为0.01的正态随机数，偏差则初始化成0。
w = torch.tensor(np.random.normal(0, 0.01, (num_inputs, 1)), dtype=torch.float32)
b = torch.zeros(1, dtype=torch.float32)

In [ ]:
# 之后的模型训练中，需要对这些参数求梯度来迭代参数的值，因此我们要让它们的requires_grad=True。
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True) 

# 定义模型

In [ ]:
# 下面是线性回归的矢量计算表达式的实现。我们使用mm函数做矩阵乘法。
def linreg(X, w, b):
    return torch.mm(X, w) + b

这个 `linreg` 函数实现了一个简单的线性回归模型的前向传播计算。下面是详细解释：

1. **函数定义**:
    ```python
    def linreg(X, w, b):
    ```
    - `X`：输入特征矩阵，形状为 `(样本数, 特征数)`。
    - `w`：权重矩阵，形状为 `(特征数, 输出数)`。
    - `b`：偏置向量，形状为 `(输出数,)`。

2. **计算线性回归模型的输出**:
    ```python
    return torch.mm(X, w) + b
    ```
    - `torch.mm(X, w)`：计算 `X` 和 `w` 的矩阵乘法。`torch.mm` 函数用于进行矩阵乘法操作，结果是一个形状为 `(样本数, 输出数)` 的张量，其中每个样本的特征向量通过权重矩阵 `w` 进行了线性变换。
    - `+ b`：将偏置向量 `b` 添加到每个样本的结果上。由于 `b` 是一个一维张量，它会被自动广播到与矩阵乘法结果相同的形状。


# 定义损失函数

In [ ]:
# 用上一节描述的平方损失来定义线性回归的损失函数。在实现中，我们需要把真实值y变形成预测值y_hat的形状。以下函数返回的结果也将和y_hat的形状相同。
def squared_loss(y_hat, y):
    # 注意这里返回的是向量, 另外, pytorch里的MSELoss并没有除以 2
    return (y_hat - y.view(y_hat.size())) ** 2 / 2

这个 `squared_loss` 函数用于计算均方误差损失（Mean Squared Error, MSE），但与 PyTorch 内置的 `MSELoss` 有所不同。下面是详细解释：

1. **函数定义**:
    ```python
    def squared_loss(y_hat, y):
    ```
    - `y_hat`：模型的预测值，通常是模型的输出。
    - `y`：真实的标签值。

2. **计算损失**:
    ```python
    return (y_hat - y.view(y_hat.size())) ** 2 / 2
    ```
    - `y.view(y_hat.size())`：调整 `y` 的形状以匹配 `y_hat` 的形状。这是因为 `y` 和 `y_hat` 可能具有不同的维度。
    - `(y_hat - y.view(y_hat.size())) ** 2`：计算预测值与真实值之间差的平方。
    - `/ 2`：将平方误差除以 2。这是因为平方误差损失函数的公式是 `(1/2) * (预测值 - 真实值)^2`。这样做是为了方便在计算梯度时使用梯度下降法，因为对损失函数的梯度是 `预测值 - 真实值`，这使得在反向传播时不会多一个因子 2。

### 注意事项

- **返回值**:
  - 这个函数返回的是每个样本的损失值的向量，而不是标量损失。这意味着你需要对这些损失值进行进一步处理，比如取平均值，来得到一个标量损失。

- **PyTorch 的 `MSELoss`**:
  - PyTorch 的 `torch.nn.MSELoss` 默认不将损失值除以 2。如果需要与 `MSELoss` 保持一致，你可以在 `squared_loss` 中去掉 `/ 2`。

### 使用示例

假设你有模型的预测值 `y_hat` 和真实标签 `y`，可以这样计算损失：

```python
# 假设 y_hat 和 y 是相同形状的 PyTorch 张量
loss = squared_loss(y_hat, y)

# 如果需要得到标量损失，可以计算所有样本的平均损失
mean_loss = loss.mean()
```

这样你就得到了每个样本的损失值，并可以通过 `.mean()` 得到所有样本的平均损失。

# 定义优化算法

In [ ]:
# 以下的sgd函数实现了上一节中介绍的小批量随机梯度下降算法。它通过不断迭代模型参数来优化损失函数。这里自动求梯度模块计算得来的梯度是一个批量样本的梯度和。我们将它除以批量大小来得到平均值。

def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size # 注意这里更改param时用的param.data

这个 `sgd` 函数实现了梯度下降优化算法中的随机梯度下降（SGD）更新规则。以下是详细解释：

1. **函数定义**:
    ```python
    def sgd(params, lr, batch_size):
    ```
    - `params`：模型的所有参数（通常是 PyTorch 张量）的列表。
    - `lr`：学习率，控制每次更新的步长。
    - `batch_size`：当前批次的样本数量，用于标准化梯度。

2. **更新每个参数**:
    ```python
    for param in params:
        param.data -= lr * param.grad / batch_size # 注意这里更改param时用的param.dat
    ```
    - `for param in params`：遍历模型的所有参数。
    - `param.data`：访问参数的实际数据（而不是梯度）。这是因为直接更改 `param` 会影响到其计算图，而 `.data` 允许我们直接修改数据而不影响计算图。
    - `param.grad`：访问参数的梯度，这些梯度是通过反向传播计算得出的。
    - `lr * param.grad / batch_size`：计算每个参数的更新量。学习率 `lr` 和梯度 `param.grad` 结合，除以 `batch_size` 是为了标准化梯度，使其与批次大小一致。

### 详细说明

- **梯度下降更新规则**:
    - 梯度下降算法通过调整参数以最小化损失函数。更新规则为：`参数 = 参数 - 学习率 * 梯度`。这里 `梯度` 是损失函数关于参数的导数。
    - `param.data -= lr * param.grad / batch_size` 这行代码按照梯度下降的规则更新参数。`lr` 控制更新的幅度，而 `param.grad` 是每个参数的梯度。

- **为什么使用 `param.data`**:
    - `param.data` 是一个不包含计算图的张量，它允许直接修改参数的值。如果直接使用 `param` 进行修改，可能会影响计算图，导致不希望的副作用。
    - `.data` 使得梯度更新不会影响到 PyTorch 的自动求导机制，从而避免了不必要的计算图更新。

### 使用示例

假设你有一组模型参数和学习率，你可以用这个 `sgd` 函数来更新模型参数：

```python
# 假设 params 是你的模型参数列表
# lr 是学习率
# batch_size 是当前批次的大小

sgd(params, lr, batch_size)
```

这样就会根据当前的梯度和学习率更新模型的参数。

# 训练模型

当然，可以用数学公式表示线性回归的训练流程。这个流程包括模型定义、损失计算、梯度计算和参数更新。以下是详细的公式表示：

### 1. 模型定义

线性回归模型的预测值 $\hat{y}$ 为：

$$
\hat{y} = Xw + b
$$

其中：
- $X$ 是输入特征矩阵，形状为 $m \times d$。
- $w$ 是权重向量，形状为 $d \times 1$。
- $b$ 是偏置，标量。

### 2. 损失函数

均方误差（MSE）损失函数定义为：

$$
L = \frac{1}{2m} \sum_{i=1}^{m} (y_i - \hat{y}_i)^2
$$

其中：
- $m$ 是批量大小。
- $y_i$ 是第 $i$ 个样本的真实标签。
- $\hat{y}_i$ 是第 $i$ 个样本的预测值。

### 3. 计算梯度

#### 对权重 $w$ 的梯度

1. 损失函数相对于预测值 $\hat{y}_i$ 的梯度为：

   $$
   \frac{\partial L}{\partial \hat{y}_i} = \hat{y}_i - y_i
   $$

2. 损失函数相对于权重 $w$ 的梯度为：

   $$
   \frac{\partial \hat{y}_i}{\partial w} = X_i
   $$

   所以：

   $$
   \frac{\partial L}{\partial w} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i) \cdot X_i
   $$

   可以简化为：

   $$
   \frac{\partial L}{\partial w} = \frac{1}{m} X^T (Xw + b - y)
   $$

#### 对偏置 $b$ 的梯度

1. 损失函数相对于偏置 $b$ 的梯度为：

   $$
   \frac{\partial L}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i)
   $$

   可以简化为：

   $$
   \frac{\partial L}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} (X_i w + b - y_i)
   $$

### 4. 参数更新

- **更新权重 $w$**:

  $$
  w = w - \eta \frac{\partial L}{\partial w}
  $$

- **更新偏置 $b$**:

  $$
  b = b - \eta \frac{\partial L}{\partial b}
  $$

其中 $\eta$ 是学习率，控制每次更新的步长。

### 总结

- **模型预测**: $\hat{y} = Xw + b$
- **损失函数**: $L = \frac{1}{2m} \sum_{i=1}^{m} (y_i - \hat{y}_i)^2$
- **梯度计算**:
  - 对权重 $w$: $\frac{\partial L}{\partial w} = \frac{1}{m} X^T (Xw + b - y)$
  - 对偏置 $b$: $\frac{\partial L}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} (X_i w + b - y_i)$
- **参数更新**:
  - $w = w - \eta \frac{\partial L}{\partial w}$
  - $b = b - \eta \frac{\partial L}{\partial b}$

In [ ]:
lr = 0.03  # 学习率，控制每次参数更新的步长
num_epochs = 3  # 训练的轮数
net = linreg  # 定义的模型函数，这里是 `linreg`
loss = squared_loss  # 定义的损失函数，这里是 `squared_loss`

for epoch in range(num_epochs):  # 训练模型一共需要num_epochs个迭代周期
    # 在每一个迭代周期中，会使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）。X
    # 和y分别是小批量样本的特征和标签
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y).sum()  # l是有关小批量X和y的损失
        l.backward()  # 小批量的损失对模型参数求梯度 （这里的梯度即为w列表对应的al/aw列表。相当于是对l求所有参数的偏导，然后再用 参数列表.grad 取出偏导列表即梯度值。）
        sgd([w, b], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数

        # 不要忘了梯度清零
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))

# 确实，在实际的训练过程中，通常我们会监控损失函数的值来判断模型的训练情况，而不是仅仅依赖预设的轮数。

这段代码用于训练一个线性回归模型，使用了自定义的损失函数和随机梯度下降（SGD）优化方法。以下是详细解释：

1. **初始化参数**:
    ```python
    lr = 0.03
    num_epochs = 3
    net = linreg
    loss = squared_loss
    ```
    - `lr`：学习率，控制每次参数更新的步长。
    - `num_epochs`：训练的轮数，这里设置为 3。
    - `net`：定义的模型函数，这里是 `linreg`。
    - `loss`：定义的损失函数，这里是 `squared_loss`。

2. **训练过程**:
    ```python
    for epoch in range(num_epochs):  # 训练模型一共需要num_epochs个迭代周期
    ```
    - 外层循环迭代 `num_epochs` 次，每次迭代代表一个训练周期（epoch）。

3. **遍历每个小批量数据**:
    ```python
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y).sum()  # l是有关小批量X和y的损失
        l.backward()  # 小批量的损失对模型参数求梯度
        sgd([w, b], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数

        # 不要忘了梯度清零
        w.grad.data.zero_()
        b.grad.data.zero_()
    ```
    - `data_iter(batch_size, features, labels)`：按批量大小迭代训练数据。
    - `net(X, w, b)`：计算当前批量数据的模型输出。
    - `loss(net(X, w, b), y).sum()`：计算当前批量数据的损失并求和。
    - `l.backward()`：通过反向传播计算损失对模型参数的梯度。
    - `sgd([w, b], lr, batch_size)`：使用随机梯度下降算法更新模型参数 `w` 和 `b`。
    - `w.grad.data.zero_()` 和 `b.grad.data.zero_()`：清零模型参数的梯度，以便下一个批次的计算。

4. **计算并打印训练集上的损失**:
    ```python
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))
    ```
    - `loss(net(features, w, b), labels)`：计算整个训练集的损失。
    - `train_l.mean().item()`：计算平均损失并转换为 Python 标量。
    - 打印当前周期的损失值，以便跟踪训练进展。

### 注意事项

- **`l.backward()` 和 `sgd([w, b], lr, batch_size)`**：`l.backward()` 计算梯度，`sgd` 函数更新参数。这两个步骤在每个批次数据后执行，确保每个批次的梯度都用于更新参数。
- **梯度清零**：在每个批次训练后清零梯度，防止梯度累积。
- **`train_l.mean().item()`**：获取训练集上的平均损失，帮助了解模型的训练效果。

这段代码实现了一个基本的训练过程，通过多次迭代训练数据，更新模型参数，以减少损失函数值，从而训练出一个更好的线性回归模型。

好的，我来用更简单的语言解释一下这部分代码的每一步：

### 代码片段解释

```python
for X, y in data_iter(batch_size, features, labels):
    l = loss(net(X, w, b), y).sum()  # l是有关小批量X和y的损失
    l.backward()  # 小批量的损失对模型参数求梯度
    sgd([w, b], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数

    # 不要忘了梯度清零
    w.grad.data.zero_()
    b.grad.data.zero_()
```

### 详细解释

1. **`for X, y in data_iter(batch_size, features, labels):`**
   - 这行代码从 `data_iter` 函数中获取一批训练数据。`data_iter` 会按批量大小（`batch_size`）将整个数据集分成若干小批量。
   - `X` 是当前批次的特征数据（比如 10 个样本的特征），`y` 是当前批次的标签（这些样本的真实值）。

2. **`l = loss(net(X, w, b), y).sum()`**
   - `net(X, w, b)` 使用当前的模型（`linreg`）来计算预测结果。`X` 是输入数据，`w` 是权重，`b` 是偏置。
   - `loss(net(X, w, b), y)` 计算模型预测结果和真实标签 `y` 之间的损失。这里用的是你定义的 `squared_loss` 函数。
   - `.sum()` 将批量中的所有样本的损失加在一起。这是因为 `loss` 函数返回的是一个包含所有样本损失的张量，我们将这些损失加在一起以获得总损失。

3. **`l.backward()`**
   - 计算损失对模型参数的梯度。`l.backward()` 是 PyTorch 的方法，它会自动计算所有模型参数的梯度，用于后续的参数更新。

4. **`sgd([w, b], lr, batch_size)`**
   - 使用随机梯度下降（SGD）来更新模型的参数。`sgd` 函数将会基于计算得到的梯度来调整权重 `w` 和偏置 `b`，以减少损失。
   - `lr` 是学习率，控制每次更新的步长。

5. **`w.grad.data.zero_()` 和 `b.grad.data.zero_()`**
   - 每次参数更新后，需要清零梯度。否则，梯度会被累积（累加到之前的梯度上），这会影响模型的训练效果。
   - `w.grad.data.zero_()` 将权重 `w` 的梯度清零，`b.grad.data.zero_()` 将偏置 `b` 的梯度清零。

### 总结

这个代码块的作用是：

- **获取数据**：从数据集中获取一批数据（特征 `X` 和标签 `y`）。
- **计算损失**：计算当前批次数据的预测损失。
- **计算梯度**：根据损失计算梯度。
- **更新参数**：使用 SGD 更新模型的权重和偏置。
- **清零梯度**：在进行下一批次的训练之前清零梯度，以免干扰。

这样做可以使得模型逐步改善，通过多次迭代训练数据来提高预测准确性。

在训练过程中，计算的梯度是相对于模型参数（如权重 `w` 和偏置 `b`）的。这些梯度表示了损失函数相对于每个参数的变化率，指导如何调整这些参数以最小化损失。下面详细解释这个过程：

### 计算梯度的过程

1. **计算损失**:
   ```python
   l = loss(net(X, w, b), y).sum()
   ```
   - 这里 `l` 是当前批次的总损失。它是模型预测值和真实标签 `y` 之间的误差。

2. **反向传播（计算梯度）**:
   ```python
   l.backward()
   ```
   - `l.backward()` 会计算损失 `l` 相对于模型参数（权重 `w` 和偏置 `b`）的梯度。
   - 在 PyTorch 中，`backward()` 会自动计算损失函数对每个模型参数的梯度。这些梯度告诉你，如果调整模型参数（如 `w` 和 `b`），损失 `l` 将如何变化。

### 为什么计算梯度相对于模型参数

1. **优化目标**:
   - 训练模型的目标是最小化损失函数。通过调整模型参数（`w` 和 `b`），我们希望减少预测误差。
   - 梯度提供了一个方向，告诉我们如何改变参数来减少损失。例如，正梯度意味着增加参数值会增加损失，而负梯度则意味着减少参数值会减少损失。

2. **梯度下降算法**:
   - 随机梯度下降（SGD）是优化算法的一种，使用梯度信息来调整参数：
     ```python
     sgd([w, b], lr, batch_size)
     ```
   - `sgd` 函数根据梯度和学习率来更新参数。这里使用的是梯度下降的规则：
     ```python
     param.data -= lr * param.grad / batch_size
     ```
   - 这里 `param.grad` 是梯度，`lr` 是学习率。通过这种方式，模型参数 `w` 和 `b` 被调整，以便减少损失函数的值。

3. **清零梯度**:
   ```python
   w.grad.data.zero_()
   b.grad.data.zero_()
   ```
   - 计算梯度后，需清零梯度以便下一次计算。否则，梯度会在每个批次之间累积，这可能导致不正确的梯度更新。

### 总结

- 计算的梯度是相对于模型的参数（如权重 `w` 和偏置 `b`）的。
- 梯度表示了损失函数关于这些参数的变化率，指引如何调整参数以最小化损失。
- 梯度下降算法使用这些梯度信息来更新参数，从而优化模型的性能。

In [ ]:
print(true_w, '\n', w)
print(true_b, '\n', b)
